In [1]:
import tensorflow as tf

2025-09-02 12:14:48.133424: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
from tensorflow.keras import Sequential, layers
model = Sequential(
    layers.input(shape(28,28,1)), # 3차원 이미지 1장의 입력
    layers.Conv2D(),
    layers.MaxPooling2D(),
    layers.Conv2D(), 
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(128, activation='relu'), # 분류 계층
    layers.Dense(10, activation='softmax') # 출력 계층
)